# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operaciona` de `Arenales, Armentano, Morabito e Yanass`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínimas e máximas de cada componente da liga).

Composição(%) e custo(R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.005 | 0.9 | 0.09 |
| Silício | 0.14 | - | 0.27 |
| Custo | 90 | 180 | 25 |

Composição(%) mínima e máxima:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 0.095 |
|Silício | 0.19 | 0.2 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Carrega Dados

In [ ]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
    }

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

Cria modelo

In [ ]:
# Criando o modelo
model = Model(sense = MINIMIZE, solver_name = CBC)

# Criando as variáveis
l = model.add_var(var_type = CONTINUOUS, name = 'l', lb = 0.0)
g = model.add_var(var_type = CONTINUOUS, name = 'g', lb = 0.0)
s = model.add_var(var_type = CONTINUOUS, name = 's', lb = 0.0)

# Criando a função objetivo
model.objective = 90*l + 180*g + 25*s

# Restrições
c_quantity = a['l']['c']*l + a['g']['c']*g + a['s']['c']*s
s_quantity = a['l']['s']*l + a['g']['s']*g + a['s']['s']*s
model += c_quantity >= 0
model += c_quantity <= 0.95
model += s_quantity >= 1.9
model += s_quantity <= 2
model += l + g + s == 10

model.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 l + 180 g + 25 s
Subject To
constr(0):  0.00500 l + 0.90000 g + 0.09000 s >= -0
constr(1):  0.00500 l + 0.90000 g + 0.09000 s <= 0.95000
constr(2):  0.14000 l + 0.27000 s >= 1.90000
constr(3):  0.14000 l + 0.27000 s <= 2
constr(4):  l + g + s = 10
Bounds
End



Executa

In [ ]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")
  
  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
l = 5.38
g = 0.00
s = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [ ]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [ ]:
# Adicionando novas restrições ao modelo
model += l <= e['l']
model += g <= e['g']
model += s <= e['s']

model.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 l + 180 g + 25 s
Subject To
constr(0):  0.00500 l + 0.90000 g + 0.09000 s >= -0
constr(1):  0.00500 l + 0.90000 g + 0.09000 s <= 0.95000
constr(2):  0.14000 l + 0.27000 s >= 1.90000
constr(3):  0.14000 l + 0.27000 s <= 2
constr(4):  l + g + s = 10
constr(5):  l <= 5
constr(6):  g <= 5
constr(7):  s <= 12
Bounds
End



Executa

In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
l = 5.00
g = 0.19
s = 4.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e sua composição mínima e máxima é dada na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 0.4 |
|Silício | 0.12 | 0.19 |


### Código

Carrega dados

In [ ]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

In [ ]:
model = # implementa modelo

model.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

SyntaxError: ignored

Executa

In [ ]:
solve(model)